# Task 3: Named Entity Recognition and Sentiment Analysis using spaCy

## Objective
Perform Named Entity Recognition (NER) and sentiment analysis on Amazon product reviews using spaCy.

## Goals
1. Load and preprocess Amazon product review data
2. Perform Named Entity Recognition to extract product names and brands
3. Analyze sentiment (positive/negative) using rule-based approach
4. Visualize and interpret the results
5. Demonstrate spaCy's capabilities for NLP tasks


## Step 1: Import Required Libraries and Setup


In [1]:
import spacy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
import re
import warnings
warnings.filterwarnings('ignore')

# Download and load spaCy model (run this if not already installed)
# !python -m spacy download en_core_web_sm

# Load the English model
try:
    nlp = spacy.load("en_core_web_sm")
    print("✅ spaCy English model loaded successfully!")
except OSError:
    print("❌ spaCy English model not found. Please run: python -m spacy download en_core_web_sm")
    print("For now, we'll create sample data to demonstrate the concepts.")

# Set style for visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print(f"spaCy version: {spacy.__version__}")
print(f"Model: en_core_web_sm")


✅ spaCy English model loaded successfully!
spaCy version: 3.8.7
Model: en_core_web_sm


## Step 2: Create Sample Amazon Review Data


In [2]:
# Create sample Amazon product reviews for demonstration
sample_reviews = [
    "I absolutely love this iPhone 15 Pro! The camera quality is amazing and the battery life lasts all day. Apple really outdid themselves with this one.",
    "The Samsung Galaxy S24 Ultra is fantastic. The display is crystal clear and the S Pen works perfectly. Highly recommend this phone!",
    "This MacBook Pro M3 is incredible. The performance is blazing fast and the build quality is top-notch. Worth every penny.",
    "I'm disappointed with this Dell XPS laptop. The battery drains too quickly and the keyboard feels cheap. Not worth the money.",
    "The Sony WH-1000XM5 headphones are amazing! The noise cancellation is perfect and the sound quality is outstanding. Sony did a great job.",
    "This Logitech MX Master 3 mouse is the best I've ever used. The ergonomics are perfect and the precision is incredible. Logitech makes quality products.",
    "I hate this cheap Android tablet. It's slow, the screen is terrible, and it crashes constantly. Complete waste of money.",
    "The Microsoft Surface Pro 9 is excellent for work. The pen input is smooth and the performance is great. Microsoft really improved this model.",
    "This HP printer is a nightmare. It constantly jams and the print quality is awful. I regret buying this piece of junk.",
    "The Canon EOS R5 camera is phenomenal. The image quality is professional-grade and the autofocus is lightning fast. Canon never disappoints.",
    "I love my new Tesla Model Y! The autopilot feature is incredible and the acceleration is mind-blowing. Tesla is revolutionizing the industry.",
    "This cheap Chinese smartphone is terrible. The camera is blurry, the battery dies quickly, and it's full of bugs. Avoid at all costs.",
    "The Apple AirPods Pro 2 are perfect. The noise cancellation is amazing and the fit is comfortable. Apple's audio products are always top quality.",
    "I'm not impressed with this Lenovo ThinkPad. The keyboard is mushy and the trackpad is unresponsive. Expected better from Lenovo.",
    "The Google Pixel 8 Pro has an incredible camera. The AI features are impressive and the software is smooth. Google really stepped up their game."
]

# Create DataFrame
df_reviews = pd.DataFrame({
    'review_text': sample_reviews,
    'review_id': range(1, len(sample_reviews) + 1)
})

print("Sample Amazon Reviews Dataset:")
print(f"Number of reviews: {len(df_reviews)}")
print("\nFirst few reviews:")
for i, review in enumerate(df_reviews['review_text'][:3]):
    print(f"{i+1}. {review}")
    print()


Sample Amazon Reviews Dataset:
Number of reviews: 15

First few reviews:
1. I absolutely love this iPhone 15 Pro! The camera quality is amazing and the battery life lasts all day. Apple really outdid themselves with this one.

2. The Samsung Galaxy S24 Ultra is fantastic. The display is crystal clear and the S Pen works perfectly. Highly recommend this phone!

3. This MacBook Pro M3 is incredible. The performance is blazing fast and the build quality is top-notch. Worth every penny.



## Step 3: Named Entity Recognition (NER)


In [ ]:
def extract_entities(review_text):
    """Extract named entities from review text using spaCy"""
    doc = nlp(review_text)
    
    entities = {
        'products': [],
        'brands': [],
        'organizations': [],
        'all_entities': []
    }
    
    for ent in doc.ents:
        entity_info = {
            'text': ent.text,
            'label': ent.label_,
            'start': ent.start_char,
            'end': ent.end_char,
            'confidence': ent._.prob if hasattr(ent._, 'prob') else None
        }
        
        entities['all_entities'].append(entity_info)
        
        # Categorize entities
        if ent.label_ in ['PRODUCT', 'WORK_OF_ART']:
            entities['products'].append(ent.text)
        elif ent.label_ in ['ORG', 'PERSON']:
            entities['brands'].append(ent.text)
        elif ent.label_ == 'ORG':
            entities['organizations'].append(ent.text)
    
    return entities

# Process all reviews
print("Processing reviews for Named Entity Recognition...")
df_reviews['entities'] = df_reviews['review_text'].apply(extract_entities)

# Extract specific entity types
df_reviews['products'] = df_reviews['entities'].apply(lambda x: x['products'])
df_reviews['brands'] = df_reviews['entities'].apply(lambda x: x['brands'])
df_reviews['all_entities'] = df_reviews['entities'].apply(lambda x: x['all_entities'])

print("✅ NER processing completed!")
print(f"Processed {len(df_reviews)} reviews")

# Display sample results
print("\n=== SAMPLE NER RESULTS ===")
for i in range(3):
    print(f"\nReview {i+1}:")
    print(f"Text: {df_reviews.iloc[i]['review_text']}")
    print(f"Products: {df_reviews.iloc[i]['products']}")
    print(f"Brands: {df_reviews.iloc[i]['brands']}")
    print(f"All Entities: {[(ent['text'], ent['label_']) for ent in df_reviews.iloc[i]['all_entities']]}")


Processing reviews for Named Entity Recognition...
✅ NER processing completed!
Processed 15 reviews

=== SAMPLE NER RESULTS ===

Review 1:
Text: I absolutely love this iPhone 15 Pro! The camera quality is amazing and the battery life lasts all day. Apple really outdid themselves with this one.
Products: []
Brands: []


KeyError: 'label_'

## Step 4: Sentiment Analysis using Rule-based Approach


In [ ]:
def rule_based_sentiment(review_text):
    """Perform rule-based sentiment analysis using keyword matching"""
    
    # Define positive and negative keywords
    positive_words = [
        'love', 'amazing', 'fantastic', 'incredible', 'excellent', 'perfect', 'great', 'awesome',
        'outstanding', 'brilliant', 'wonderful', 'superb', 'magnificent', 'phenomenal', 'outstanding',
        'best', 'top', 'quality', 'impressive', 'smooth', 'fast', 'clear', 'crystal', 'blazing',
        'revolutionary', 'step up', 'outdid', 'never disappoints', 'always', 'worth', 'recommend'
    ]
    
    negative_words = [
        'hate', 'terrible', 'awful', 'disappointed', 'disappointing', 'bad', 'worst', 'horrible',
        'nightmare', 'junk', 'waste', 'regret', 'avoid', 'cheap', 'slow', 'terrible', 'blurry',
        'bugs', 'crashes', 'jams', 'mushy', 'unresponsive', 'drains', 'constantly', 'not impressed'
    ]
    
    # Convert to lowercase for case-insensitive matching
    text_lower = review_text.lower()
    
    # Count positive and negative words
    positive_count = sum(1 for word in positive_words if word in text_lower)
    negative_count = sum(1 for word in negative_words if word in text_lower)
    
    # Calculate sentiment score
    sentiment_score = positive_count - negative_count
    
    # Determine sentiment label
    if sentiment_score > 0:
        sentiment = 'positive'
    elif sentiment_score < 0:
        sentiment = 'negative'
    else:
        sentiment = 'neutral'
    
    return {
        'sentiment': sentiment,
        'score': sentiment_score,
        'positive_count': positive_count,
        'negative_count': negative_count
    }

# Apply sentiment analysis to all reviews
print("Performing rule-based sentiment analysis...")
df_reviews['sentiment_analysis'] = df_reviews['review_text'].apply(rule_based_sentiment)

# Extract sentiment components
df_reviews['sentiment'] = df_reviews['sentiment_analysis'].apply(lambda x: x['sentiment'])
df_reviews['sentiment_score'] = df_reviews['sentiment_analysis'].apply(lambda x: x['score'])
df_reviews['positive_words'] = df_reviews['sentiment_analysis'].apply(lambda x: x['positive_count'])
df_reviews['negative_words'] = df_reviews['sentiment_analysis'].apply(lambda x: x['negative_count'])

print("✅ Sentiment analysis completed!")

# Display sample results
print("\n=== SAMPLE SENTIMENT RESULTS ===")
for i in range(5):
    print(f"\nReview {i+1}:")
    print(f"Text: {df_reviews.iloc[i]['review_text'][:100]}...")
    print(f"Sentiment: {df_reviews.iloc[i]['sentiment']} (Score: {df_reviews.iloc[i]['sentiment_score']})")
    print(f"Positive words: {df_reviews.iloc[i]['positive_words']}, Negative words: {df_reviews.iloc[i]['negative_words']}")


## Step 5: Visualize and Analyze Results


In [ ]:
# Create comprehensive visualizations
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Sentiment Distribution
sentiment_counts = df_reviews['sentiment'].value_counts()
axes[0, 0].pie(sentiment_counts.values, labels=sentiment_counts.index, autopct='%1.1f%%', startangle=90)
axes[0, 0].set_title('Sentiment Distribution')

# 2. Sentiment Scores Distribution
axes[0, 1].hist(df_reviews['sentiment_score'], bins=10, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 1].set_title('Sentiment Score Distribution')
axes[0, 1].set_xlabel('Sentiment Score')
axes[0, 1].set_ylabel('Frequency')

# 3. Most Common Products
all_products = [product for products in df_reviews['products'] for product in products]
product_counts = Counter(all_products)
if product_counts:
    top_products = dict(product_counts.most_common(8))
    axes[0, 2].bar(range(len(top_products)), list(top_products.values()))
    axes[0, 2].set_title('Most Mentioned Products')
    axes[0, 2].set_xticks(range(len(top_products)))
    axes[0, 2].set_xticklabels(list(top_products.keys()), rotation=45, ha='right')

# 4. Most Common Brands
all_brands = [brand for brands in df_reviews['brands'] for brand in brands]
brand_counts = Counter(all_brands)
if brand_counts:
    top_brands = dict(brand_counts.most_common(8))
    axes[1, 0].bar(range(len(top_brands)), list(top_brands.values()), color='lightcoral')
    axes[1, 0].set_title('Most Mentioned Brands')
    axes[1, 0].set_xticks(range(len(top_brands)))
    axes[1, 0].set_xticklabels(list(top_brands.keys()), rotation=45, ha='right')

# 5. Sentiment vs Word Count
axes[1, 1].scatter(df_reviews['positive_words'], df_reviews['negative_words'], 
                   c=df_reviews['sentiment_score'], cmap='RdYlGn', alpha=0.7)
axes[1, 1].set_title('Positive vs Negative Word Count')
axes[1, 1].set_xlabel('Positive Words')
axes[1, 1].set_ylabel('Negative Words')

# 6. Entity Types Distribution
all_entities = [ent for entities in df_reviews['all_entities'] for ent in entities]
entity_labels = [ent['label_'] for ent in all_entities]
entity_label_counts = Counter(entity_labels)
if entity_label_counts:
    axes[1, 2].pie(entity_label_counts.values(), labels=entity_label_counts.keys(), 
                   autopct='%1.1f%%', startangle=90)
    axes[1, 2].set_title('Entity Types Distribution')

plt.tight_layout()
plt.show()

# Print summary statistics
print("=== ANALYSIS SUMMARY ===")
print(f"\n📊 Sentiment Analysis:")
print(f"   • Positive reviews: {sum(df_reviews['sentiment'] == 'positive')}")
print(f"   • Negative reviews: {sum(df_reviews['sentiment'] == 'negative')}")
print(f"   • Neutral reviews: {sum(df_reviews['sentiment'] == 'neutral')}")
print(f"   • Average sentiment score: {df_reviews['sentiment_score'].mean():.2f}")

print(f"\n🏷️ Named Entity Recognition:")
print(f"   • Total entities found: {len(all_entities)}")
print(f"   • Unique products: {len(set(all_products))}")
print(f"   • Unique brands: {len(set(all_brands))}")
print(f"   • Entity types: {list(entity_label_counts.keys())}")

print(f"\n📈 Most Common Entities:")
if product_counts:
    print(f"   • Top products: {list(product_counts.most_common(3))}")
if brand_counts:
    print(f"   • Top brands: {list(brand_counts.most_common(3))}")


## Step 6: Detailed Entity Analysis and Examples


In [ ]:
# Display detailed entity analysis for each review
print("=== DETAILED ENTITY AND SENTIMENT ANALYSIS ===")
print("=" * 60)

for idx, row in df_reviews.iterrows():
    print(f"\n📝 Review {idx + 1}:")
    print(f"Text: {row['review_text']}")
    print(f"Sentiment: {row['sentiment']} (Score: {row['sentiment_score']})")
    
    if row['all_entities']:
        print("Entities found:")
        for ent in row['all_entities']:
            print(f"  • {ent['text']} ({ent['label_']})")
    else:
        print("No entities detected")
    
    print("-" * 40)

# Create a comprehensive results DataFrame
results_df = df_reviews[['review_id', 'review_text', 'sentiment', 'sentiment_score', 
                        'products', 'brands', 'all_entities']].copy()

print(f"\n=== FINAL RESULTS DATAFRAME ===")
print(f"Shape: {results_df.shape}")
print(f"\nColumns: {list(results_df.columns)}")
print(f"\nFirst few rows:")
print(results_df.head())

# Save results to CSV
results_df.to_csv('amazon_reviews_analysis.csv', index=False)
print(f"\n✅ Results saved to 'amazon_reviews_analysis.csv'")

print(f"\n=== TASK 3 COMPLETE: spaCy NER and Sentiment Analysis ===")
print(f"✅ All objectives achieved:")
print(f"   ✓ Amazon review data loaded and preprocessed")
print(f"   ✓ Named Entity Recognition performed to extract products and brands")
print(f"   ✓ Rule-based sentiment analysis implemented")
print(f"   ✓ Results visualized and analyzed")
print(f"   ✓ spaCy capabilities demonstrated for NLP tasks")
